In [1]:
# Gaussian Elimination

from typing import List, Tuple

Matrix = List[List[float]]
Vector = List[float]


def gaussian_elimination_solve(A: Matrix, b: Vector, *, pivot: bool = True) -> Vector:

    n = len(A)
    if n == 0 or any(len(row) != n for row in A):
        raise ValueError("A must be a non-empty square matrix.")
    if len(b) != n:
        raise ValueError("b must have the same length as A's dimension.")

    M = [row[:] for row in A]
    rhs = b[:]

    for k in range(n):
        if pivot:
            max_row = max(range(k, n), key=lambda i: abs(M[i][k]))
            if abs(M[max_row][k]) < 1e-15:
                raise ValueError("Matrix is singular or nearly singular (pivot ~ 0).")
            if max_row != k:
                M[k], M[max_row] = M[max_row], M[k]
                rhs[k], rhs[max_row] = rhs[max_row], rhs[k]
        else:
            if abs(M[k][k]) < 1e-15:
                raise ValueError("Zero pivot encountered (enable pivoting or reorder equations).")

        pivot_val = M[k][k]
        for i in range(k + 1, n):
            factor = M[i][k] / pivot_val
            M[i][k] = 0.0
            for j in range(k + 1, n):
                M[i][j] -= factor * M[k][j]
            rhs[i] -= factor * rhs[k]

    x = [0.0] * n
    for i in range(n - 1, -1, -1):
        s = rhs[i]
        for j in range(i + 1, n):
            s -= M[i][j] * x[j]
        if abs(M[i][i]) < 1e-15:
            raise ValueError("Matrix is singular or nearly singular (diag ~ 0).")
        x[i] = s / M[i][i]

    return x


In [2]:
# LU Decomposition

def mat_vec_mul(A: Matrix, x: Vector) -> Vector:
    return [sum(A[i][j] * x[j] for j in range(len(x))) for i in range(len(A))]


def identity(n: int) -> Matrix:
    return [[1.0 if i == j else 0.0 for j in range(n)] for i in range(n)]


def forward_substitution(L: Matrix, b: Vector) -> Vector:
    n = len(L)
    y = [0.0] * n
    for i in range(n):
        s = b[i]
        for j in range(i):
            s -= L[i][j] * y[j]
        if abs(L[i][i]) < 1e-15:
            raise ValueError("Singular L (zero diagonal).")
        y[i] = s / L[i][i]
    return y


def back_substitution(U: Matrix, y: Vector) -> Vector:
    n = len(U)
    x = [0.0] * n
    for i in range(n - 1, -1, -1):
        s = y[i]
        for j in range(i + 1, n):
            s -= U[i][j] * x[j]
        if abs(U[i][i]) < 1e-15:
            raise ValueError("Singular U (zero diagonal).")
        x[i] = s / U[i][i]
    return x


def lu_decomposition_partial_pivot(A: Matrix) -> Tuple[Matrix, Matrix, Matrix]:
    n = len(A)
    if n == 0 or any(len(row) != n for row in A):
        raise ValueError("A must be a non-empty square matrix.")

    U = [row[:] for row in A]
    L = identity(n)
    P = identity(n)

    for k in range(n):
        pivot_row = max(range(k, n), key=lambda i: abs(U[i][k]))
        if abs(U[pivot_row][k]) < 1e-15:
            raise ValueError("Matrix is singular or nearly singular (pivot ~ 0).")

        if pivot_row != k:
            U[k], U[pivot_row] = U[pivot_row], U[k]
            P[k], P[pivot_row] = P[pivot_row], P[k]
            for j in range(k):
                L[k][j], L[pivot_row][j] = L[pivot_row][j], L[k][j]

        for i in range(k + 1, n):
            L[i][k] = U[i][k] / U[k][k]
            U[i][k] = 0.0
            for j in range(k + 1, n):
                U[i][j] -= L[i][k] * U[k][j]

    return L, U, P


def lu_solve(A: Matrix, b: Vector) -> Vector:
    L, U, P = lu_decomposition_partial_pivot(A)
    pb = mat_vec_mul(P, b)
    y = forward_substitution(L, pb)
    x = back_substitution(U, y)
    return x


In [3]:
# Test

if __name__ == "__main__":
    A = [
        [2.0, 1.0, -1.0],
        [-3.0, -1.0, 2.0],
        [-2.0, 1.0, 2.0]
    ]
    b = [8.0, -11.0, -3.0]

    x_ge = gaussian_elimination_solve(A, b, pivot=True)
    x_lu = lu_solve(A, b)

    print("Gaussian Elimination x =", x_ge)
    print("LU Solve x             =", x_lu)


Gaussian Elimination x = [2.0, 3.0000000000000004, -0.9999999999999999]
LU Solve x             = [2.0, 3.0000000000000004, -0.9999999999999999]
